# MinerU → RAG (Windowized) Pipeline with Rerank


In [2]:
%pip install mineru[core]>=2.1.9


Note: you may need to restart the kernel to use updated packages.


  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.14.4 requires dill<0.3.8,>=0.3.0, but you have dill 0.4.0 which is incompatible.
llama-index-readers-file 0.4.11 requires pandas<2.3.0, but you have pandas 2.3.1 which is incompatible.
tensorflow-intel 2.14.0 requires wrapt<1.15,>=1.11.0, but you have wrapt 1.17.2 which is incompatible.


In [3]:
%pip install -U --no-cache-dir numpy pandas


Note: you may need to restart the kernel to use updated packages.


In [17]:
# 1) Setup & config
import os, json, re, hashlib, time, glob
from pathlib import Path
from collections import defaultdict

# repo layout (adjust as needed)
BASE = Path.cwd()
DATAS_DIR = BASE.parent / "data" / "PDFs"                       
MINERU_CONTENT_DIR = BASE / "data_base_json_content"
MINERU_PAGE_DIR    = BASE / "data_base_json_page_content"
CHUNKS_JSON        = BASE / "all_pdf_page_chunks.json"


MAX_PDFS_TO_PARSE    = 2        # for quick tests; set None for all
FORCE_REPARSE        = False    # True to force re-run MinerU even if outputs exist
ENABLE_IMAGE_CAPTION = False    # keep False for speed/cost; you can enable later

# env (LLM / embeddings / rerank)
# Expect these set in your .env or environment
for k in ["LOCAL_BASE_URL","LOCAL_TEXT_MODEL","LOCAL_EMBEDDING_MODEL","LOCAL_API_KEY"]:
    print(k, "=", "***" if os.getenv(k) else None)
    

print("DATAS_DIR          :", DATAS_DIR)
print("MINERU_CONTENT_DIR :", MINERU_CONTENT_DIR)
print("MINERU_PAGE_DIR    :", MINERU_PAGE_DIR)
print("CHUNKS_JSON        :", CHUNKS_JSON)

# page numbering expected by the evaluation (1 or 0)
PAGE_BASE = 1  # set to 1 if the competition uses 1-based pages



LOCAL_BASE_URL = ***
LOCAL_TEXT_MODEL = ***
LOCAL_EMBEDDING_MODEL = ***
LOCAL_API_KEY = ***
DATAS_DIR          : d:\Datawhale\Multimodal-RAG-Competitions\data\PDFs
MINERU_CONTENT_DIR : d:\Datawhale\Multimodal-RAG-Competitions\notebook\data_base_json_content
MINERU_PAGE_DIR    : d:\Datawhale\Multimodal-RAG-Competitions\notebook\data_base_json_page_content
CHUNKS_JSON        : d:\Datawhale\Multimodal-RAG-Competitions\notebook\all_pdf_page_chunks.json


## 2) Import the MinerU parse entry
We’ll try to import `parse_doc` from either `do_parse.py` or `mineru_parse_pdf.py` (whichever you have).


In [9]:
import os
from pathlib import Path
import json
from collections import defaultdict
import asyncio
import sys
sys.path.insert(0, str(Path.cwd().parent))

from image_utils.async_image_analysis import AsyncImageAnalysis

In [21]:
def parse_all_pdfs(datas_dir, output_base_dir,MAX_PDFS_TO_PARSE):
    """
    步骤1：解析所有PDF，输出内容到 data_base_json_content/
    """
    from mineru_parse_pdf import do_parse
    datas_dir = Path(datas_dir)
    output_base_dir = Path(output_base_dir)
    pdf_files = list(datas_dir.rglob('*.pdf'))
    if MAX_PDFS_TO_PARSE:
        pdf_files = pdf_files[:MAX_PDFS_TO_PARSE]
    if not pdf_files:
        print(f"未找到PDF文件于: {datas_dir}")
        return
    for pdf_path in pdf_files:
        file_name = pdf_path.stem
        with open(pdf_path, "rb") as f:
            pdf_bytes = f.read()
        output_dir = output_base_dir / file_name
        output_dir.mkdir(parents=True, exist_ok=True)
        do_parse(
            output_dir=str(output_dir),
            pdf_file_names=[file_name],
            pdf_bytes_list=[pdf_bytes],
            p_lang_list=["ch"],
            backend="pipeline",
            f_draw_layout_bbox=False,
            f_draw_span_bbox=False,
            f_dump_md=False,
            f_dump_middle_json=False,
            f_dump_model_output=False,
            f_dump_orig_pdf=False,
            f_dump_content_list=True
        )
        print(f"已输出: {output_dir / 'auto' / (file_name + '_content_list.json')}")

In [11]:
def group_by_page(content_list):
    pages = defaultdict(list)
    for item in content_list:
        page_idx = item.get('page_idx', 0)
        pages[page_idx].append(item)
    return dict(pages)

In [12]:
def item_to_markdown(item, enable_image_caption=True):
    """
    enable_image_caption: 是否启用多模态视觉分析（图片caption补全），默认True。
    """
    # 默认API参数：硅基流动Qwen/Qwen2.5-VL-32B-Instruct
    vision_provider = "guiji"
    vision_model = "Pro/Qwen/Qwen2.5-VL-7B-Instruct"
    vision_api_key = os.getenv("LOCAL_API_KEY")
    vision_base_url = os.getenv("LOCAL_BASE_URL")
    
    if item['type'] == 'text':
        level = item.get('text_level', 0)
        text = item.get('text', '')
        if level == 1:
            return f"# {text}\n\n"
        elif level == 2:
            return f"## {text}\n\n"
        else:
            return f"{text}\n\n"
    elif item['type'] == 'image':
        captions = item.get('image_caption', [])
        caption = captions[0] if captions else ''
        img_path = item.get('img_path', '')
        # 如果没有caption，且允许视觉分析，调用多模态API补全
        if enable_image_caption and not caption and img_path and os.path.exists(img_path):
            try:
                loop = asyncio.new_event_loop()
                asyncio.set_event_loop(loop)
                async def get_caption():
                    async with AsyncImageAnalysis(
                        provider=vision_provider,
                        api_key=vision_api_key,
                        base_url=vision_base_url,
                        vision_model=vision_model
                    ) as analyzer:
                        result = await analyzer.analyze_image(local_image_path=img_path)
                        return result.get('title') or result.get('description') or ''
                caption = loop.run_until_complete(get_caption())
                loop.close()
                if caption:
                    item['image_caption'] = [caption]
            except Exception as e:
                print(f"图片解释失败: {img_path}, {e}")
        md = f"![{caption}]({img_path})\n"
        return md + "\n"
    elif item['type'] == 'table':
        captions = item.get('table_caption', [])
        caption = captions[0] if captions else ''
        table_html = item.get('table_body', '')
        img_path = item.get('img_path', '')
        md = ''
        if caption:
            md += f"**{caption}**\n"
        if img_path:
            md += f"![{caption}]({img_path})\n"
        md += f"{table_html}\n\n"
        return md
    else:
        return '\n'

In [13]:
def assemble_pages_to_markdown(pages):
    page_md = {}
    for page_idx in sorted(pages.keys()):
        md = ''
        for item in pages[page_idx]:
            md += item_to_markdown(item, enable_image_caption=True)
        page_md[page_idx] = md
    return page_md
    return page_md

In [14]:
def process_all_pdfs_to_page_json(input_base_dir, output_base_dir):
    """
    步骤2：将 content_list.json 转为 page_content.json
    """
    input_base_dir = Path(input_base_dir)
    output_base_dir = Path(output_base_dir)
    pdf_dirs = [d for d in input_base_dir.iterdir() if d.is_dir()]
    for pdf_dir in pdf_dirs:
        file_name = pdf_dir.name
        json_path = pdf_dir / 'auto' / f'{file_name}_content_list.json'
        if not json_path.exists():
            sub_dir = pdf_dir / file_name
            json_path2 = sub_dir / 'auto' / f'{file_name}_content_list.json'
            if json_path2.exists():
                json_path = json_path2
            else:
                print(f"未找到: {json_path} 也未找到: {json_path2}")
                continue
        with open(json_path, 'r', encoding='utf-8') as f:
            content_list = json.load(f)
        pages = group_by_page(content_list)
        page_md = assemble_pages_to_markdown(pages)
        output_dir = output_base_dir / file_name
        output_dir.mkdir(parents=True, exist_ok=True)
        output_json_path = output_dir / f'{file_name}_page_content.json'
        with open(output_json_path, 'w', encoding='utf-8') as f:
            json.dump(page_md, f, ensure_ascii=False, indent=2)
        print(f"已输出: {output_json_path}")

In [15]:
def process_page_content_to_chunks(input_base_dir, output_json_path):
    """
    步骤3：将 page_content.json 合并为 all_pdf_page_chunks.json
    """
    input_base_dir = Path(input_base_dir)
    all_chunks = []
    for pdf_dir in input_base_dir.iterdir():
        if not pdf_dir.is_dir():
            continue
        file_name = pdf_dir.name
        page_content_path = pdf_dir / f"{file_name}_page_content.json"
        if not page_content_path.exists():
            sub_dir = pdf_dir / file_name
            page_content_path2 = sub_dir / f"{file_name}_page_content.json"
            if page_content_path2.exists():
                page_content_path = page_content_path2
            else:
                print(f"未找到: {page_content_path} 也未找到: {page_content_path2}")
                continue
        with open(page_content_path, 'r', encoding='utf-8') as f:
            page_dict = json.load(f)
        for page_idx, content in page_dict.items():
            chunk = {
                "id": f"{file_name}_page_{page_idx}",
                "content": content,
                "metadata": {
                    "page": page_idx,
                    "file_name": file_name + ".pdf"
                }
            }
            all_chunks.append(chunk)
    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(all_chunks, f, ensure_ascii=False, indent=2)
    print(f"已输出: {output_json_path}")

In [ ]:
# 1) Setup & config
import os, json, re, hashlib, time, glob
from pathlib import Path
from collections import defaultdict

# repo layout (adjust as needed)
BASE = Path.cwd()
DATAS_DIR = BASE.parent / "data" / "PDFs"                       
MINERU_CONTENT_DIR = BASE / "data_base_json_content"
MINERU_PAGE_DIR    = BASE / "data_base_json_page_content"
CHUNKS_JSON        = BASE / "all_pdf_page_chunks.json"


MAX_PDFS_TO_PARSE    = 2        # for quick tests; set None for all
FORCE_REPARSE        = False    # True to force re-run MinerU even if outputs exist
ENABLE_IMAGE_CAPTION = False    # keep False for speed/cost; you can enable later

# env (LLM / embeddings / rerank)
# Expect these set in your .env or environment
for k in ["LOCAL_BASE_URL","LOCAL_TEXT_MODEL","LOCAL_EMBEDDING_MODEL","LOCAL_API_KEY"]:
    print(k, "=", "***" if os.getenv(k) else None)
    

print("DATAS_DIR          :", DATAS_DIR)
print("MINERU_CONTENT_DIR :", MINERU_CONTENT_DIR)
print("MINERU_PAGE_DIR    :", MINERU_PAGE_DIR)
print("CHUNKS_JSON        :", CHUNKS_JSON)

# page numbering expected by the evaluation (1 or 0)
PAGE_BASE = 1  # set to 1 if the competition uses 1-based pages



In [18]:
pdfs = sorted(DATAS_DIR.rglob("*.pdf"))
assert pdfs, f"No PDFs found under {DATAS_DIR}. Put some test PDFs there."

if MAX_PDFS_TO_PARSE:
    pdfs = pdfs[:MAX_PDFS_TO_PARSE]

In [22]:
parse_all_pdfs(DATAS_DIR, MINERU_CONTENT_DIR,2)

ImportError: DLL load failed while importing onnxruntime_pybind11_state: A dynamic link library (DLL) initialization routine failed.

In [ ]:
def main():
    base_dir = Path(__file__).parent
    datas_dir = base_dir / 'datas'
    content_dir = base_dir / 'data_base_json_content'
    page_dir = base_dir / 'data_base_json_page_content'
    chunk_json_path = base_dir / 'all_pdf_page_chunks.json'
    # 步骤1：PDF → content_list.json
    parse_all_pdfs(datas_dir, content_dir)
    # 步骤2：content_list.json → page_content.json
    process_all_pdfs_to_page_json(content_dir, page_dir)
    # 步骤3：page_content.json → all_pdf_page_chunks.json
    process_page_content_to_chunks(page_dir, chunk_json_path)
    print("全部处理完成！")

if __name__ == '__main__':
    main()